In [4]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import math
from tqdm import tqdm

In [1]:
class Model:
    def __init__(self, train_data, train_label, num):
        self.train_data = train_data   # 训练集数据
        self.train_label = train_label # 训练集标记
        self.num = num                 # 多少个基本分类器
        # m为样本容量，n为特征数量
        m, n = np.shape(train_data)
        self.m = m                     # m为样本容量
        self.n = n                     # n为特征数量
        self.w = np.full(m, fill_value=1/m)
        
    def create_single_boosting_tree(self) :
        
    def create_boosting_tree(self) :
        for N in tqdm(self.num):
            tree = self.create_single_boosting_tree()
    
        

SyntaxError: unexpected EOF while parsing (797973202.py, line 3)

In [5]:
np.full(10, fill_value=1/10)

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])